In [1]:
import folium as f
import pandas as pd

### Note:

This is a tutorial from https://blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9

Load a map using the Map method. ```zoom_start``` is the initial zoom level on the map. ```tiles``` is the style of map to choose from; refer documentation.

In [143]:
folium_map = f.Map(location=[40.738, -73.98],
                  zoom_start=13,
                  tiles="CartoDB dark_matter")
folium_map

Add marker to map. The circle marker includes a ```radius``` in meters.

In [144]:
marker = f.CircleMarker(location=[40.738, -73.98])
# marker = f.Marker(location=[3.1390, 101.6869])
#marker = f.Circle(location=[3.1390, 101.6869], radius=3000)
marker.add_to(folium_map)

In [145]:
folium_map

In [38]:
bike_data = pd.read_csv('201610-citibike-tripdata.csv')
bike_data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,328,2016-10-01 00:00:07,2016-10-01 00:05:35,471,Grand St & Havemeyer St,40.712868,-73.956981,3077,Stagg St & Union Ave,40.708771,-73.950953,25254,Subscriber,1992.0,1
1,398,2016-10-01 00:00:11,2016-10-01 00:06:49,3147,E 85 St & 3 Ave,40.778012,-73.954071,3140,1 Ave & E 78 St,40.771404,-73.953517,17810,Subscriber,1988.0,2
2,430,2016-10-01 00:00:14,2016-10-01 00:07:25,345,W 13 St & 6 Ave,40.736494,-73.997044,470,W 20 St & 8 Ave,40.743453,-74.000040,20940,Subscriber,1965.0,1
3,351,2016-10-01 00:00:21,2016-10-01 00:06:12,3307,West End Ave & W 94 St,40.794165,-73.974124,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,19086,Subscriber,1993.0,1
4,2693,2016-10-01 00:00:21,2016-10-01 00:45:15,3428,8 Ave & W 16 St,40.740983,-74.001702,3323,W 106 St & Central Park West,40.798186,-73.960591,26502,Subscriber,1991.0,1


In [43]:
bike_data['Start Time'] = pd.to_datetime(bike_data['Start Time'])
bike_data['Stop Time'] = pd.to_datetime(bike_data['Stop Time'])

In [99]:
# Both lines achieve same output.
# bike_data['hour'] = bike_data['Start Time'].map(lambda x: x.hour)
bike_data['Start Hour'] = bike_data['Start Time'].dt.hour
bike_data['Stop Hour'] = bike_data['Stop Time'].dt.hour

In [100]:
bike_data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Start Hour,Stop Hour
0,328,2016-10-01 00:00:07,2016-10-01 00:05:35,471,Grand St & Havemeyer St,40.712868,-73.956981,3077,Stagg St & Union Ave,40.708771,-73.950953,25254,Subscriber,1992.0,1,0,0
1,398,2016-10-01 00:00:11,2016-10-01 00:06:49,3147,E 85 St & 3 Ave,40.778012,-73.954071,3140,1 Ave & E 78 St,40.771404,-73.953517,17810,Subscriber,1988.0,2,0,0
2,430,2016-10-01 00:00:14,2016-10-01 00:07:25,345,W 13 St & 6 Ave,40.736494,-73.997044,470,W 20 St & 8 Ave,40.743453,-74.000040,20940,Subscriber,1965.0,1,0,0
3,351,2016-10-01 00:00:21,2016-10-01 00:06:12,3307,West End Ave & W 94 St,40.794165,-73.974124,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,19086,Subscriber,1993.0,1,0,0
4,2693,2016-10-01 00:00:21,2016-10-01 00:45:15,3428,8 Ave & W 16 St,40.740983,-74.001702,3323,W 106 St & Central Park West,40.798186,-73.960591,26502,Subscriber,1991.0,1,0,0


In [172]:
def aggregate_data(time):
    # Groupby station ID and return the first row of each group. Objective is just to get all the station names and locations from the dataset.
    locations = bike_data.groupby('Start Station ID').first()
    # Select only the three required columns
    locations = locations[["Start Station Latitude", "Start Station Longitude", "Start Station Name"]]
    
    # Pick an hour of the day to observe the number of bike departures during that time
    subset_start = bike_data[bike_data['Start Hour'] == time]
    departure_counts = subset_start.groupby('Start Station ID').count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["Departure Count"]
    
    # Pick an hour of the day to observe the number of bike arrivals during that time
    subset_stop = bike_data[bike_data['Stop Hour'] == time]
    arrival_counts = subset_stop.groupby('End Station ID').count()
    arrival_counts = arrival_counts.iloc[:,[0]]
    arrival_counts.columns= ["Arrival Count"]

    trip_counts = departure_counts.join(locations).join(arrival_counts)

    trip_counts = trip_counts.dropna()

    trip_counts['Arrival Count'] = trip_counts['Arrival Count'].astype('int64')
    
    return trip_counts

In [173]:
def plot_net_departures(df):
    folium_map = f.Map(location=[40.738, -73.98],
                  zoom_start=13,
                  tiles="CartoDB dark_matter")
    
    for index, row in df.iterrows():
        net_departures = row['Departure Count']-row['Arrival Count']
        radius = net_departures/20
        if net_departures>0:
            color="#E37222" # tangerine
        else:
            color="#0A8A9F" # teal
        
        popup_text = '{}<br> total departures: {}<br> total arrivals: {}<br> net departures {}'.format(row["Start Station Name"],
                          row["Departure Count"],
                          row["Arrival Count"],
                          net_departures)

        f.CircleMarker(location=(row['Start Station Latitude'],
                                  row['Start Station Longitude']),
                        radius=radius,
                        color=color,fill=True, popup=popup_text).add_to(folium_map)
    return folium_map

In [174]:
trip_at_9 = aggregate_data(9)
plot_net_departures(trip_at_9)

In [175]:
trip_at_18 = aggregate_data(18)
plot_net_departures(trip_at_18)